<a href="https://colab.research.google.com/github/boskidisanalysis/Movies_Recomendation/blob/main/MovieLens_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 25.5 MB/s 
     |████████████████████████████████| 85 kB 6.3 MB/s 
     |████████████████████████████████| 182 kB 77.5 MB/s 
     |████████████████████████████████| 7.6 MB 69.3 MB/s 
     |████████████████████████████████| 1.3 MB 68.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=5934cc4490780c67f56522b401b7f912db0811cac42da869e36995b6cd9a61dd
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


# Language Model Sentence Embeddings

In [2]:
def get_zip_file(url):
  # importing the requests module
  import requests
  print('Downloading started')
  

  # Downloading the file by sending the request to the URL
  req = requests.get(url)

  # Split URL to get the file name
  filename = url.split('/')[-1]

  # Writing the file to the local file system
  with open(filename,'wb') as output_file:
    output_file.write(req.content)
  print('Downloading Completed')

In [3]:
url = 'https://files.grouplens.org/datasets/movielens/ml-25m.zip'
get_zip_file(url)

In [4]:
!unzip ml-25m.zip

Archive:  ml-25m.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


In [5]:
import pandas as pd 
import numpy as np
from transformers import AutoTokenizer, AutoModel

In [6]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/LaBSE')
model = AutoModel.from_pretrained('sentence-transformers/LaBSE')
data = pd.read_csv('/content/ml-25m/movies.csv')

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/804 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

In [7]:
data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
import re
def remove_pars(x):
  x = str(x)
  return re.sub('[()]',"",x)

In [9]:
titles = [remove_pars(i) for i in data['title']]

In [10]:
titles[:10]

['Toy Story 1995',
 'Jumanji 1995',
 'Grumpier Old Men 1995',
 'Waiting to Exhale 1995',
 'Father of the Bride Part II 1995',
 'Heat 1995',
 'Sabrina 1995',
 'Tom and Huck 1995',
 'Sudden Death 1995',
 'GoldenEye 1995']

In [11]:
def remove_pipes(x):
  x=str(x)
  return re.sub('\|', " ",x)

In [12]:
genres = [remove_pipes(i) for i in data['genres']]

In [13]:
def remove_nulls(a,b,i):
  string_m = a[i] + ' '+ b[i]
  return re.sub('\(no genres listed\)',"", string_m)

In [14]:
input_string = [remove_nulls(titles, genres, i) for i in range(len(genres)) ]

In [15]:
input_string[:10]

['Toy Story 1995 Adventure Animation Children Comedy Fantasy',
 'Jumanji 1995 Adventure Children Fantasy',
 'Grumpier Old Men 1995 Comedy Romance',
 'Waiting to Exhale 1995 Comedy Drama Romance',
 'Father of the Bride Part II 1995 Comedy',
 'Heat 1995 Action Crime Thriller',
 'Sabrina 1995 Comedy Romance',
 'Tom and Huck 1995 Adventure Children',
 'Sudden Death 1995 Action',
 'GoldenEye 1995 Action Adventure Thriller']

In [17]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

In [ ]:
embeddings_list = []
for _, i in enumerate(input_string):
  encoded_input = tokenizer(i, padding=True, truncation=True, max_length=64, return_tensors='pt').to(device)
  with torch.inference_mode(): 
    model_output = model(**encoded_input)
  embeddings = model_output.pooler_output
  embeddings = torch.nn.functional.normalize(embeddings)
  embeddings_list.append(embeddings)
  if _ %10000 == 0:
    print(str(_))  

0
10000
20000
30000
40000
50000
60000


In [ ]:
embeddings_list_tensors = [i.cpu()[0].numpy() for i in embeddings_list]

In [ ]:
embeddings = pd.DataFrame(np.vstack(embeddings_list_tensors))
embeddings.to_csv('embedding_data.csv')

In [ ]:
embeddings = pd.read_csv('/content/drive/MyDrive/embedding_data.csv',index_col=0)

In [ ]:
embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.006538,-0.042496,0.014313,0.023817,-0.056576,0.047191,0.004182,0.011531,-0.010112,-0.026587,...,-0.035090,-0.022659,-0.017100,-0.039019,-0.056206,-0.006634,-0.034283,-0.031559,0.035495,0.041886
1,-0.008588,-0.030542,0.001161,0.017724,-0.051122,0.065883,0.001737,0.023441,-0.029678,-0.048359,...,-0.001177,0.046209,0.021217,-0.040132,-0.066025,0.035727,-0.022926,-0.064467,0.030721,0.009855
2,-0.003954,-0.058009,0.000429,0.031793,-0.055129,0.041872,0.007370,0.004464,0.017399,-0.052856,...,-0.024539,0.047893,0.008946,-0.027328,-0.027161,0.045568,-0.020122,-0.057999,-0.041704,0.008740
3,0.015047,-0.061407,-0.016938,-0.013356,-0.024377,0.032822,-0.039581,0.018516,-0.003029,-0.022575,...,-0.047106,0.044167,-0.026389,-0.038091,-0.038319,0.036390,-0.037342,-0.063318,0.016386,0.024849
4,-0.012277,-0.061566,-0.006264,-0.001114,-0.035620,0.005099,-0.065697,0.063327,0.074066,-0.015134,...,-0.007887,0.035019,-0.001221,-0.014433,-0.050452,0.050904,-0.040659,-0.008459,0.023693,0.021617


# Approximate Nearest Neighbor Candidate Generation

In [ ]:
!pip install tensorflow-recommenders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 4.1 MB/s 


In [ ]:
!pip install scann

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.5 MB 7.4 MB/s 
     |████████████████████████████████| 588.3 MB 21 kB/s 
     |████████████████████████████████| 6.0 MB 53.8 MB/s 
     |████████████████████████████████| 439 kB 63.9 MB/s 
     |████████████████████████████████| 1.7 MB 57.7 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.

In [ ]:
import tensorflow as tf 
import tensorflow_recommenders as tfrs

In [ ]:
item_tensor = tf.convert_to_tensor(embeddings, dtype=tf.float32)

In [ ]:
scann = tfrs.layers.factorized_top_k.ScaNN(num_leaves=1000,
                                           num_leaves_to_search=100,
                                           k=round(np.sqrt(len(item_tensor))))
scann.index(item_tensor)

In [ ]:
search = "Comedy films for Christmas" #@param {type:"string"}

In [ ]:
encoded_input = tokenizer(search, padding=True, truncation=True, max_length=64, return_tensors = 'pt').to(device)
with torch.inference_mode():
  model_output = model(**encoded_input)
query = model_output.pooler_output
query = torch.nn.functional.normalize(query)

In [ ]:
test_case = scann(np.array(query.cpu()))

In [ ]:
test_case[0][0][0:10].numpy()

array([0.6197646 , 0.6063759 , 0.58653545, 0.5725466 , 0.57201517,
       0.5703147 , 0.56798476, 0.5673812 , 0.5666513 , 0.56088406],
      dtype=float32)

In [ ]:
data.iloc[test_case[1].numpy()[0]][0:10]

,movieId,title,genres
25351,123530,The Christmas Party (2009),Comedy
57672,196897,Christmas Crush (2012),Comedy|Drama
42922,164907,A Star for Christmas (2012),Comedy|Drama|Romance
45183,169804,Internet - O Filme (2017),Comedy
11969,56158,This Christmas (2007),Comedy|Drama|Romance
44336,168010,A Christmas Wish (2011),Children|Comedy|Drama
49948,179953,A Bad Moms Christmas (2017),Comedy
62143,208046,Täydellinen joulu (2019),Comedy
45311,170105,Saattokeikka (2017),Comedy|Drama
13587,70344,Cold Souls (2009),Comedy|Drama


In [ ]:
def prediction(text:str):
  encoded_input = tokenizer(text, padding=True, truncation=True, max_length=64, return_tensors = 'pt').to(device)
  with torch.inference_mode():
    model_output = model(**encoded_input)
  query = model_output.pooler_output
  query = torch.nn.functional.normalize(query)
  test_case = scann(np.array(query.cpu()))
  return data.iloc[test_case[1].numpy()[0]][0:10]

In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.6 MB 8.9 MB/s 
     |████████████████████████████████| 84 kB 4.1 MB/s 
     |████████████████████████████████| 106 kB 72.0 MB/s 
     |████████████████████████████████| 213 kB 72.7 MB/s 
     |████████████████████████████████| 278 kB 68.2 MB/s 
     |████████████████████████████████| 56 kB 4.9 MB/s 
     |████████████████████████████████| 55 kB 4.2 MB/s 
     |████████████████████████████████| 2.3 MB 48.3 MB/s 
     |████████████████████████████████| 54 kB 3.9 MB/s 
     |████████████████████████████████| 84 kB 3.7 MB/s 
     |████████████████████████████████| 64 kB 3.5 MB/s 
     |████████████████████████████████| 80 kB 10.8 MB/s 
     |████████████████████████████████| 68 kB 8.0 MB/s 
     |████████████████████████████████| 68 kB 7.9 MB/s 
     |████████████████████████████████| 68 kB 8.2 MB/s 
     |████████████████████████████████| 68 kB 7.5 

In [ ]:
import gradio as gr

# We instantiate the Textbox class
textbox = gr.Textbox(label="Type your search here:", placeholder="Search for a film", lines=2)

gr.Interface(fn=prediction, inputs=textbox, outputs="dataframe").launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

# MiniLM Sentence-Transformers

In [16]:
from sentence_transformers import SentenceTransformer


embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2',)
movies_embeddings = embedder.encode(input_string, convert_to_tensor = True)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [18]:
top_k = min(10, len(input_string))

In [20]:
movies_embeddings.device

device(type='cuda', index=0)

In [22]:
query = 'A comedy for Christmas ' #@param {type: 'string'}

In [47]:
from sentence_transformers import util
query_embedding = embedder.encode(query, convert_to_tensor=True)

# We use cosine-similarity and torch.topk to find the highest 5 scores
cos_scores = util.cos_sim(query_embedding, movies_embeddings)[0]
top_results = torch.topk(cos_scores, k=top_k)

print("\n\n======================\n\n")
print("Query:", query)
print("\nTop 10 most similar sentences in corpus:")

for score, idx in zip(top_results[0].cpu(), top_results[1].cpu()):
  print(data['title'].iloc[idx.numpy()]," || ",data['genres'].iloc[idx.numpy()] ,' || ', f"(Score: {score:.4f})")

# """
# # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
# hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
# hits = hits[0]      #Get the hits for the first query
# for hit in hits:
#   print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
# """





Query: A comedy for Christmas 

Top 10 most similar sentences in corpus:
Almost Christmas (2016)  ||  Comedy  ||  (Score: 0.8062)
Surviving Christmas (2004)  ||  Comedy  ||  (Score: 0.8017)
Chasing Christmas (2005)  ||  Comedy|Fantasy  ||  (Score: 0.7935)
A Merry Friggin' Christmas (2014)  ||  Comedy  ||  (Score: 0.7886)
5 Star Christmas (2018)  ||  Comedy  ||  (Score: 0.7852)
The Ultimate Christmas Present (2000)  ||  Children|Comedy  ||  (Score: 0.7847)
Christmas Do-Over (2006)  ||  Children|Comedy|Fantasy  ||  (Score: 0.7812)
Christmas All Over Again (2016)  ||  Children|Comedy  ||  (Score: 0.7796)
Christmas Story, A (1983)  ||  Children|Comedy  ||  (Score: 0.7767)
A Cookie Cutter Christmas (2014)  ||  Comedy  ||  (Score: 0.7738)


In [52]:
results = data.iloc[top_results[1].cpu()]

In [63]:
results['score'] = top_results[0].cpu().numpy()

<ipython-input-63-de9ea3d0de2a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['score'] = top_results[0].cpu().numpy()


In [69]:
top_results[0].cpu().numpy().round(5)

array([0.8062 , 0.80171, 0.79353, 0.78863, 0.78519, 0.78469, 0.78119,
       0.77963, 0.77668, 0.77384], dtype=float32)

In [64]:
results

,movieId,title,genres,score
43284,165681,Almost Christmas (2016),Comedy,0.806197
8231,8946,Surviving Christmas (2004),Comedy,0.801706
23856,119703,Chasing Christmas (2005),Comedy|Fantasy,0.793527
23927,119964,A Merry Friggin' Christmas (2014),Comedy,0.788626
57840,197275,5 Star Christmas (2018),Comedy,0.785192
30017,135256,The Ultimate Christmas Present (2000),Children|Comedy,0.784689
43202,165501,Christmas Do-Over (2006),Children|Comedy|Fantasy,0.781187
57654,196855,Christmas All Over Again (2016),Children|Comedy,0.779633
2712,2804,"Christmas Story, A (1983)",Children|Comedy,0.776679
36158,149578,A Cookie Cutter Christmas (2014),Comedy,0.773844


In [38]:
from sentence_transformers import util
query_embedding = embedder.encode(query, convert_to_tensor=True)

# We use cosine-similarity and torch.topk to find the highest 5 scores
cos_scores = util.cos_sim(query_embedding, movies_embeddings)[0]
top_results = torch.topk(cos_scores, k=top_k)

print("\n\n======================\n\n")
print("Query:", query)
print("\nTop 10 most similar sentences in corpus:")

for score, idx in zip(top_results[0].cpu(), top_results[1].cpu()):
  print(idx.numpy())





Query: A comedy for Christmas 

Top 10 most similar sentences in corpus:
43284
8231
23856
23927
57840
30017
43202
57654
2712
36158


In [42]:
data.iloc[43284]

movieId                     165681
title      Almost Christmas (2016)
genres                      Comedy
Name: 43284, dtype: object

In [24]:
from sentence_transformers import util
query_embedding = embedder.encode(query, convert_to_tensor=True)

# We use cosine-similarity and torch.topk to find the highest 5 scores
cos_scores = util.cos_sim(query_embedding, movies_embeddings)[0]
top_results = torch.topk(cos_scores, k=top_k)

print("\n\n======================\n\n")
print("Query:", query)
print("\nTop 10 most similar sentences in corpus:")





Query: A comedy for Christmas 

Top 10 most similar sentences in corpus:


In [32]:
top_results

torch.return_types.topk(
values=tensor([0.8062, 0.8017, 0.7935, 0.7886, 0.7852, 0.7847, 0.7812, 0.7796, 0.7767,
        0.7738], device='cuda:0'),
indices=tensor([43284,  8231, 23856, 23927, 57840, 30017, 43202, 57654,  2712, 36158],
       device='cuda:0'))

In [31]:
top_results[0].device

device(type='cuda', index=0)

In [57]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.8 MB 6.8 MB/s 
     |████████████████████████████████| 278 kB 70.4 MB/s 
     |████████████████████████████████| 55 kB 4.3 MB/s 
     |████████████████████████████████| 106 kB 64.6 MB/s 
     |████████████████████████████████| 213 kB 79.2 MB/s 
     |████████████████████████████████| 54 kB 3.7 MB/s 
     |████████████████████████████████| 2.3 MB 60.5 MB/s 
     |████████████████████████████████| 84 kB 4.4 MB/s 
     |████████████████████████████████| 56 kB 5.6 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 
     |████████████████████████████████| 64 kB 3.6 MB/s 
     |████████████████████████████████| 80 kB 11.1 MB/s 
     |████████████████████████████████| 68 kB 9.2 MB/s 
     |████████████████████████████████| 68 kB 8.9 MB/s 
     |████████████████████████████████| 68 kB 8.2 MB/s 
     |████████████████████████████████| 68 kB 7.9 

In [86]:
def prediction(query:str):
  query_embedding = embedder.encode(query, convert_to_tensor=True)

  # We use cosine-similarity and torch.topk to find the highest 5 scores
  cos_scores = util.cos_sim(query_embedding, movies_embeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)
  results = data.iloc[top_results[1].cpu()].reset_index()
  results['score'] = top_results[0].cpu().numpy().round(4)
  return results[['title','genres','score']]


In [87]:
prediction("A good christmas movie")

,title,genres,score
0,A Bad Moms Christmas (2017),Comedy,0.7279
1,A Different Kind of Christmas (1996),Drama,0.7102
2,"Christmas Story, A (1983)",Children|Comedy,0.7052
3,A Very Country Christmas (2017),Drama,0.6962
4,A Merry Friggin' Christmas (2014),Comedy,0.6948
5,A Christmas Visitor (2002),Drama,0.6926
6,A Cookie Cutter Christmas (2014),Comedy,0.6924
7,Christmas Carol: The Movie (2001),Animation|Children,0.6911
8,A Dogwalker's Christmas Tale (2015),Comedy,0.6899
9,The Spirit of Christmas (1995),Animation|Comedy,0.6896


In [88]:
import gradio as gr

# We instantiate the Textbox class
textbox = gr.Textbox(label="Type your search here:", placeholder="Search for a film", lines=2)

gr.Interface(fn=prediction, inputs=textbox, outputs="dataframe").launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>